In [4]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
train = pd.read_excel('/content/drive/MyDrive/Datasets/final_meth(proteomics).xlsx',index_col=0)
train.head()

,cell_name,index,chr1:10003165-10003585,chr1:100315420-100316009,chr1:100435297-100436070,chr1:100503482-100504404,chr1:10057121-10058108,chr1:100598130-100598754,chr1:100731632-100731999,chr1:100817709-100818899,chr1:10092532-10093404,chr1:101004471-101005885,chr1:101360588-101360832,chr1:101361549-101361808,chr1:101491317-101491680,chr1:101701785-101702068,chr1:101702445-101702745,chr1:10270015-10271306,chr1:1027509-1027730,chr1:104068487-104068913,chr1:10458548-10459997,chr1:1046044-1046340,chr1:10490200-10491072,chr1:1050515-1050830,chr1:1051178-1052445,chr1:10532177-10532800,chr1:10534918-10535172,chr1:10753773-10754176,chr1:10755226-10755521,chr1:10757891-10758252,chr1:107599205-107600410,chr1:10764449-10764925,chr1:107682889-107684463,chr1:108507251-108507841,chr1:10853894-10856964,chr1:108735227-108735513,chr1:108742400-108743201,chr1:109102538-109103212,chr1:109203593-109204378,chr1:109234788-109235161,...,chr9:95055762-95056299,chr9:95087949-95088606,chr9:95431812-95432671,chr9:95526459-95527825,chr9:95569429-95572255,chr9:95820744-95821526,chr9:95857703-95859132,chr9:95896007-95897016,chr9:95946147-95947835,chr9:96108466-96108992,chr9:96213340-96214953,chr9:96338040-96339836,chr9:96713326-96718186,chr9:96720586-96723189,chr9:96792652-96793436,chr9:97021464-97021967,chr9:97136589-97137656,chr9:97401286-97402067,chr9:97488818-97489172,chr9:976162-977828,chr9:97848140-97848525,chr9:98054676-98054916,chr9:98075418-98075741,chr9:98079189-98080213,chr9:98268475-98271454,chr9:98272643-98273383,chr9:98278128-98279754,chr9:98637245-98638445,chr9:98783216-98784364,chr9:99145524-99145849,chr9:99179479-99181723,chr9:99212164-99213507,chr9:99328810-99329764,chr9:99380814-99382236,chr9:99416904-99417592,chr9:99540025-99540472,chr9:99616401-99616940,chr9:99775098-99776102,chr9:99800856-99801724,chr9:99962136-99962426
0,AU565,7970376031_R03C02,0.177929,0.190773,0.164969,0.213848,0.242535,0.218838,0.168273,0.124485,0.146088,0.307858,0.205958,0.148976,0.210555,0.253641,0.086934,0.193070,0.921250,0.157768,0.130472,0.757466,0.317818,0.373853,0.259566,0.080981,0.247547,0.146795,0.515792,0.762865,0.268358,0.388833,0.136182,0.289008,0.176611,0.835056,0.174881,0.289434,0.539071,0.097451,...,0.090070,0.188333,0.170621,0.255697,0.288467,0.436971,0.126562,0.482742,0.772923,0.787916,0.143449,0.403087,0.111230,0.293658,0.267954,0.251573,0.313893,0.841703,0.204294,0.115008,0.819592,0.525097,0.575341,0.470626,0.117331,0.272886,0.139505,0.233938,0.180470,0.723128,0.163983,0.126866,0.129329,0.301235,0.474329,0.249210,0.179306,0.280969,0.143087,0.101365
15,HCC1143,8221932018_R01C01,0.214136,0.178600,0.178052,0.198967,0.364215,0.258488,0.263512,0.106214,0.238094,0.575203,0.258321,0.174136,0.237026,0.444943,0.263330,0.221230,0.920528,0.182537,0.158440,0.863808,0.292629,0.472320,0.411455,0.107303,0.302131,0.220532,0.119538,0.715037,0.380793,0.834516,0.295935,0.409930,0.201151,0.871229,0.218645,0.361974,0.379844,0.129081,...,0.098640,0.188729,0.192366,0.333110,0.398967,0.482346,0.115091,0.284594,0.538190,0.921220,0.151643,0.293932,0.535941,0.705892,0.321893,0.204359,0.339231,0.714174,0.320730,0.558442,0.904402,0.776713,0.639564,0.528204,0.234873,0.287027,0.402121,0.221714,0.512504,0.557001,0.177475,0.218055,0.198571,0.334596,0.558776,0.563899,0.410484,0.343744,0.261457,0.548077
16,HCC1187,7970376031_R02C02,0.208781,0.230877,0.306833,0.247515,0.358042,0.292915,0.283187,0.147705,0.204865,0.519743,0.274074,0.152832,0.222830,0.361878,0.224661,0.195332,0.902036,0.191496,0.131949,0.857881,0.294045,0.464874,0.361720,0.164411,0.320660,0.175209,0.202492,0.817014,0.489807,0.783038,0.304505,0.205483,0.199287,0.513854,0.217106,0.477082,0.548377,0.113395,...,0.101210,0.194491,0.202290,0.326427,0.341918,0.424491,0.205816,0.313638,0.284441,0.773474,0.119975,0.229909,0.596824,0.552559,0.336938,0.398083,0.318487,0.646596,0.292537,0.419834,0.863483,0.696204,0.425960,0.532611,0.293558,0.275159,0.382755,0.223433,0.464938,0.660887,0.169913,0.213462,0.151902,0.318786,0.57796

In [8]:
target=train['cell_name']

In [9]:
train.drop(['cell_name'], axis=1, inplace=True)
train.drop(['index'], axis=1, inplace=True)
train.head()

,chr1:10003165-10003585,chr1:100315420-100316009,chr1:100435297-100436070,chr1:100503482-100504404,chr1:10057121-10058108,chr1:100598130-100598754,chr1:100731632-100731999,chr1:100817709-100818899,chr1:10092532-10093404,chr1:101004471-101005885,chr1:101360588-101360832,chr1:101361549-101361808,chr1:101491317-101491680,chr1:101701785-101702068,chr1:101702445-101702745,chr1:10270015-10271306,chr1:1027509-1027730,chr1:104068487-104068913,chr1:10458548-10459997,chr1:1046044-1046340,chr1:10490200-10491072,chr1:1050515-1050830,chr1:1051178-1052445,chr1:10532177-10532800,chr1:10534918-10535172,chr1:10753773-10754176,chr1:10755226-10755521,chr1:10757891-10758252,chr1:107599205-107600410,chr1:10764449-10764925,chr1:107682889-107684463,chr1:108507251-108507841,chr1:10853894-10856964,chr1:108735227-108735513,chr1:108742400-108743201,chr1:109102538-109103212,chr1:109203593-109204378,chr1:109234788-109235161,chr1:109289018-109289822,chr1:109419544-109420038,...,chr9:95055762-95056299,chr9:95087949-95088606,chr9:95431812-95432671,chr9:95526459-95527825,chr9:95569429-95572255,chr9:95820744-95821526,chr9:95857703-95859132,chr9:95896007-95897016,chr9:95946147-95947835,chr9:96108466-96108992,chr9:96213340-96214953,chr9:96338040-96339836,chr9:96713326-96718186,chr9:96720586-96723189,chr9:96792652-96793436,chr9:97021464-97021967,chr9:97136589-97137656,chr9:97401286-97402067,chr9:97488818-97489172,chr9:976162-977828,chr9:97848140-97848525,chr9:98054676-98054916,chr9:98075418-98075741,chr9:98079189-98080213,chr9:98268475-98271454,chr9:98272643-98273383,chr9:98278128-98279754,chr9:98637245-98638445,chr9:98783216-98784364,chr9:99145524-99145849,chr9:99179479-99181723,chr9:99212164-99213507,chr9:99328810-99329764,chr9:99380814-99382236,chr9:99416904-99417592,chr9:99540025-99540472,chr9:99616401-99616940,chr9:99775098-99776102,chr9:99800856-99801724,chr9:99962136-99962426
0,0.177929,0.190773,0.164969,0.213848,0.242535,0.218838,0.168273,0.124485,0.146088,0.307858,0.205958,0.148976,0.210555,0.253641,0.086934,0.193070,0.921250,0.157768,0.130472,0.757466,0.317818,0.373853,0.259566,0.080981,0.247547,0.146795,0.515792,0.762865,0.268358,0.388833,0.136182,0.289008,0.176611,0.835056,0.174881,0.289434,0.539071,0.097451,0.136012,0.376723,...,0.090070,0.188333,0.170621,0.255697,0.288467,0.436971,0.126562,0.482742,0.772923,0.787916,0.143449,0.403087,0.111230,0.293658,0.267954,0.251573,0.313893,0.841703,0.204294,0.115008,0.819592,0.525097,0.575341,0.470626,0.117331,0.272886,0.139505,0.233938,0.180470,0.723128,0.163983,0.126866,0.129329,0.301235,0.474329,0.249210,0.179306,0.280969,0.143087,0.101365
15,0.214136,0.178600,0.178052,0.198967,0.364215,0.258488,0.263512,0.106214,0.238094,0.575203,0.258321,0.174136,0.237026,0.444943,0.263330,0.221230,0.920528,0.182537,0.158440,0.863808,0.292629,0.472320,0.411455,0.107303,0.302131,0.220532,0.119538,0.715037,0.380793,0.834516,0.295935,0.409930,0.201151,0.871229,0.218645,0.361974,0.379844,0.129081,0.283806,0.385339,...,0.098640,0.188729,0.192366,0.333110,0.398967,0.482346,0.115091,0.284594,0.538190,0.921220,0.151643,0.293932,0.535941,0.705892,0.321893,0.204359,0.339231,0.714174,0.320730,0.558442,0.904402,0.776713,0.639564,0.528204,0.234873,0.287027,0.402121,0.221714,0.512504,0.557001,0.177475,0.218055,0.198571,0.334596,0.558776,0.563899,0.410484,0.343744,0.261457,0.548077
16,0.208781,0.230877,0.306833,0.247515,0.358042,0.292915,0.283187,0.147705,0.204865,0.519743,0.274074,0.152832,0.222830,0.361878,0.224661,0.195332,0.902036,0.191496,0.131949,0.857881,0.294045,0.464874,0.361720,0.164411,0.320660,0.175209,0.202492,0.817014,0.489807,0.783038,0.304505,0.205483,0.199287,0.513854,0.217106,0.477082,0.548377,0.113395,0.221080,0.458185,...,0.101210,0.194491,0.202290,0.326427,0.341918,0.424491,0.205816,0.313638,0.284441,0.773474,0.119975,0.229909,0.596824,0.552559,0.336938,0.398083,0.318487,0.646596,0.292537,0.419834,0.863483,0.696204,0.425960,0.532611,0.293558,0.275159,0.382755,0.223433,0.464938,0.660887,0.169913,0.213462,0.151902,0.31

In [10]:
train_scaled = minmax_scale(train, axis = 0)

In [11]:
ncol = train_scaled.shape[1]
X_train, X_test, Y_train, Y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))

In [12]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Deep Autoencoder

# this is the size of our encoded representations
encoding_dim = 16

# this is our input placeholder; 784 = 28 x 28
input = Input(shape = (ncol, ))

my_epochs = 50

# "encoded" is the encoded representation of the inputs
encoded = Dense(encoding_dim * 4, activation='relu')(input)
encoded = Dense(encoding_dim * 2, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(encoding_dim * 2, activation='relu')(encoded)
decoded = Dense(encoding_dim * 4, activation='relu')(decoded)
decoded = Dense(ncol, activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input, decoded)

# Separate Encoder model

# this model maps an input to its encoded representation
encoder = Model(input, encoded)

# Separate Decoder model

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim, ))
# retrieve the layers of the autoencoder model
decoder_layer1 = autoencoder.layers[-3]
decoder_layer2 = autoencoder.layers[-2]
decoder_layer3 = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer3(decoder_layer2(decoder_layer1(encoded_input))))

# Train to reconstruct MNIST digits

# configure model to use a per-pixel binary crossentropy loss, and the Adadelta optimizer
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# prepare input data
#(x_train, _), (x_test, y_test) = mnist.load_data()
x_train, x_test, y_train, y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))
# normalize all values between 0 and 1 and flatten the 28x28 images into vectors of size 784
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

# Train autoencoder for 50 epochs

autoencoder.fit(x_train, x_train, epochs=my_epochs, batch_size=8, shuffle=True, validation_data=(x_test, x_test),
                verbose=2)

# after 100 epochs the autoencoder seems to reach a stable train/test lost value

# Visualize the reconstructed encoded representations

# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)


encoded_train = pd.DataFrame(encoder.predict(train_scaled))
encoded_train = encoded_train.add_prefix('feature_')
encoded_train.head()

K.clear_session()

(10, 14726)
(2, 14726)
Epoch 1/50
2/2 - 1s - loss: 0.6932 - val_loss: 0.6931 - 889ms/epoch - 444ms/step
Epoch 2/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 66ms/epoch - 33ms/step
Epoch 3/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 60ms/epoch - 30ms/step
Epoch 4/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 51ms/epoch - 25ms/step
Epoch 5/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 55ms/epoch - 27ms/step
Epoch 6/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 56ms/epoch - 28ms/step
Epoch 7/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 53ms/epoch - 27ms/step
Epoch 8/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 59ms/epoch - 30ms/step
Epoch 9/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 59ms/epoch - 30ms/step
Epoch 10/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 58ms/epoch - 29ms/step
Epoch 11/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 69ms/epoch - 34ms/step
Epoch 12/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 58ms/epoch - 29ms/step
Epoch 13/50
2/2 - 0s - l

In [13]:
encoded_train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,0.553441,0.0,0.000000,0.012895,0.200476,0.395362,0.000000,0.0,0.418534,0.000000,0.0,0.573506,0.115791,0.000000,0.00000,0.212664
1,1.146610,0.0,0.000000,0.460480,0.000000,0.191943,0.000000,0.0,0.162560,0.386201,0.0,0.672285,0.185293,0.053671,0.00000,0.000000
2,1.241244,0.0,0.361858,0.214426,0.000000,0.182053,0.000000,0.0,0.321434,0.000000,0.0,0.774745,0.000000,0.000000,0.00000,0.289592
3,1.064294,0.0,0.000000,0.758883,0.219702,0.463735,0.000000,0.0,0.339350,0.080710,0.0,1.063232,0.061320,0.000000,0.00000,0.057821
4,0.699088,0.0,0.000000,0.244682,0.344626,0.000000,0.000000,0.0,0.511578,0.000000,0.0,0.210379,0.233996,0.000000,0.00000,0.000000
5,0.668301,0.0,0.000000,0.583668,0.453522,0.357669,0.000000,0.0,0.515446,0.000000,0.0,0.921041,0.057368,0.000000,0.00000,0.000000
6,0.752739,0.0,0.000000,0.451828,0.228959,0.210050,0.000000,0.0,0.391444,0.000000,0.0,0.715968,0.100112,0.000000,0.00000,0.000000
7,0.613893,0.0,0.000000,0.656287,0.343658,0.178626,0.000000,0.0,0.367961,0.060751,0.0,0.345313,0.004962,0.000000,0.00000,0.000000
8,0.783685,0.0,0.000000,0.529254,0.445288,0.000000,0.000000,0.0,0.145621,0.164048,0.0,0.394503,0.000000,0.000000,0.00000,0.000000
9,0.754446,0.0,0.000000,0.766049,0.000000,0.189924,0.000000,0.0,0.146328,0.000000,0.0,0.226476,0.000000,0.000000,0.33792,0.042530


In [14]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14726)]           0         
                                                                 
 dense (Dense)               (None, 64)                942528    
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 32)                544       
                                                                 
 dense_4 (Dense)             (None, 64)                2112      
                                                                 
 dense_5 (Dense)             (None, 14726)             957190

In [15]:
encoded_train = encoded_train.loc[:, (encoded_train != 0).any(axis=0)]
encoded_train = encoded_train.drop(columns=encoded_train.columns[encoded_train.eq(0).mean()>0.8])
encoded_train.shape

(12, 9)

In [16]:
encoded_train.to_excel('/content/drive/MyDrive/Datasets/dae_meth(proteomics).xlsx', index=False)